Data generator

Generate_X_y_c:

Input:

- a: the number of rows of data you want to generate

- b: the number of rows of data you want to generate

Output:

- A 2D array with a rows and b columns

In [0]:
import random
import numpy as np
from numpy.linalg import inv

'''
array_generate_helper (n)
paramater: n (integer) --> generate number of n 
return : an array with the length of n
'''

def Array_Generate_Helper(n):
  array = []
  for i in range (0,n):
    value = random.random()*10
    array.append(value)
  return array
  

'''
function preprocess:
paramater: N(double) --> array with size 100
paramater: P(Double) --> array with size 1000
return a N*P matrix --> 2d list
'''

def Preprocess(N,P):
  nPerson_array = np.array(Array_Generate_Helper(N))
  nBrain_array  = np.array(Array_Generate_Helper(P))
  # convert the np_format array to np_matrix 
  nPerson_matrix = nPerson_array.reshape(N,1)
  nBrain_matrix = nBrain_array.reshape(1,P)
  #do the mutiply 
  result = np.dot(nPerson_matrix ,nBrain_matrix);
  # print(result.shape)
  return result


def Generate_X_y_c(a,b):
  # build up a matrix with [a,b]
  X= Preprocess(a,b)
  #build a np array with length a
  y = np.array(Array_Generate_Helper(a))
  c = np.array(Array_Generate_Helper(a))
  return X,y,c


Model 1

Input:

- m: brain data, size (NxP)


- age: vector of ages, size (Nx1)


- CI_percentage: the percentage of confidence interval you want to take

- resample_time: the time of resampling you want to perform

Output:

- The resampling result of b1 Bootstrap

- The resampling result of b1 Jackknife

- The confidence interval of b1 Bootstrap

- The confidence interval of b1 Jackknife

- The Z score of b1 Bootstrap

- The Z score of b1 Jackknife



In [0]:
from scipy import stats
from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats
import math
import scipy.linalg as la
from sklearn import linear_model
from sklearn.utils import resample

class model_1:
  def __init__(self, m,age,CI_percentage,resample_time):
        self.m = m
        self.age = age
        self.CI_percentage = CI_percentage
        self.resample_time = resample_time
        
  def Calculate_Beta_Sklearn(self,X,y):
    # using linear regression model from sklearn 
    lm = linear_model.LinearRegression()
    #fit the x,y to the model,x will be a 2d matrix and y is a array
    model = lm.fit(X,y)
    #default method to calculate the beta
    beta = model.coef_
    inter = model.intercept_
    # print(beta)
    return beta,inter

  def Resample_Sklearn(self,l,func,X,y):
    betalist = []
    interlist = []
    for i in range(0,l):
      # randomly resample the dataset with the original set with replacement
      a = resample(X,n_samples=len(X),replace=True, random_state=i)
      beta,inter = func(a,y)
      betalist.append(beta)
      interlist.append(inter)
    return betalist,interlist


  def m1_bootstrap(self,m,age):
    coeflist=[]
    interlist=[]
    for i in range(0,len(m[0])):
      mi = m[:,i].reshape(-1,1)
      result_coef, result_inter = self.Resample_Sklearn(resample_time ,self.Calculate_Beta_Sklearn,mi,age)
      coeflist.append(result_coef)
      interlist.append(result_inter)
    coeflist =np.array(coeflist)
    interlist =np.array(interlist)
    #return coeflist, interlist
    return coeflist

  def m1_jackknife(self,m,age):
    coeflist_jackknife=[]
    interlist_jackknife=[]
    for i in range(0,len(m[0])):
      mjack = m[:,i]
      resamples = jackknife_resampling(mjack)
      for ii in range(0,len(resamples)):
        resamplesi = np.append(resamples[ii],[1])
        resamplesi = resamplesi.reshape(-1,1)
        result_coef_jackknife,result_inter_jackknife= self.Calculate_Beta_Sklearn(resamplesi,age)
        coeflist_jackknife.append(result_coef_jackknife)
        interlist_jackknife.append(result_inter_jackknife)
      # result = Calculate_Beta_Numpy(mi,age)
      # coeflist.append(result)
    coeflist_jackknife =np.array(coeflist_jackknife)
    interlist_jackknife =np.array(interlist_jackknife)
    #return coeflist_jackknife,interlist_jackknife
    return coeflist_jackknife


  def calculate_bias_interval(self,array,percent):
    lower = (1-percent)/2
    upper = 1-lower
    length = array.size
    lower_index = math.ceil((length *lower))
    upper_index = math.floor((length *upper))
    
    return array[lower_index:upper_index]

  def m1_zscore(self,coeflist):
    z = stats.zscore(coeflist)
    return z


  def print_result(self,m,age,CI_percentage,resample_time):
      b1_bootstrap = self.m1_bootstrap(m,age)
      b1_jackknife = self.m1_jackknife(m,age)
      CI_b1_bootstrap = self.calculate_bias_interval(np.sort(b1_bootstrap[:,:,0].ravel()),CI_percentage)
      CI_b1_jackknife = self.calculate_bias_interval(np.sort(b1_jackknife[:,0].ravel()),CI_percentage)
      zscore_b1_bootstrap = self.m1_zscore(CI_b1_bootstrap)
      zscore_b1_jackknife = self.m1_zscore(CI_b1_jackknife)
      print("\n The resampling result of b1 Bootstrap is: " )
      print(b1_bootstrap[:,:,0].ravel())
      print("\n The resampling result of b1 Jackknife is: "  )
      print(b1_jackknife[:,0].ravel())
      print("\n The confidence interval of b1 Bootstrap is: "  )
      print(CI_b1_bootstrap.ravel())
      print("\n The confidence interval of b1 Jackknife is: "  )
      print(CI_b1_jackknife.ravel())
      print("\n The Z score of b1 Bootstrap is: " )
      print(zscore_b1_bootstrap.ravel())
      print("\n The Z score of b1 Jackknife is: " )
      print(zscore_b1_jackknife.ravel())
      return b1_bootstrap[:,:,0].ravel(),b1_jackknife[:,0].ravel(),CI_b1_bootstrap.ravel(),CI_b1_jackknife.ravel(),zscore_b1_bootstrap.ravel(),zscore_b1_jackknife.ravel()


Model 2

Input:


- m: brain data, size (NxP)


- age: vector of ages, size (Nx1)

- c: congnitive scores

- CI_percentage: the percentage of confidence interval you want to take

- resample_time: the time of resampling you want to perform

Output:

- The resampling result of c1 Bootstrap

- The resampling result of c2 Bootstrap

- The resampling result of c1 Jackknife

- The resampling result of c2 Jackknife

- The confidence interval of c1 Bootstrap

- The confidence interval of c2 Bootstrap

- The confidence interval of c1 Jackknife

- The confidence interval of c2 Jackknife

- The Z score of c1 Bootstrap

- The Z score of c2 Bootstrap

- The Z score of c1 Jackknife

- The Z score of c1 Jackknife


In [0]:

class model_2:
    def __init__(self, m,age,c,CI_percentage,resample_time):
            self.m = m
            self.age = age
            self.c =c
            self.CI_percentage = CI_percentage
            self.resample_time = resample_time

    def Calculate_Beta_Sklearn(self,X,y):
        # using linear regression model from sklearn 
        lm = linear_model.LinearRegression()
        #fit the x,y to the model,x will be a 2d matrix and y is a array
        model = lm.fit(X,y)
        #default method to calculate the beta
        beta = model.coef_
        inter = model.intercept_
        # print(beta)
        return beta,inter

    def Resample_Sklearn(self,l,func,X,y):
        betalist = []
        interlist = []
        for i in range(0,l):
          # randomly resample the dataset with the original set with replacement
          a = resample(X,n_samples=len(X),replace=True, random_state=i)
          beta,inter = func(a,y)
          betalist.append(beta)
          interlist.append(inter)
        return betalist,interlist


    def m2_bootstrap(self,m,age,c): 
          agei = age.reshape(-1,1)
          ci = c.reshape(-1,1)
          coeflist=[]
          interlist=[]
          for i in range(0,len(m[0])):
            mi = m[:,i].reshape(-1,1)
            combine = np.concatenate((agei,mi),axis=1)
            # print(combine)
            result_coef, result_inter = self.Resample_Sklearn(resample_time ,self.Calculate_Beta_Sklearn,combine,ci)
            coeflist.append(result_coef)
            interlist.append(result_inter)
          coeflist =np.array(coeflist)
          interlist =np.array(interlist)
          return coeflist

    def m2_jackknife(self,m,age,c):

          coeflist_jackknife=[]
          interlist_jackknife=[]
          ages = jackknife_resampling(age)
          cs = jackknife_resampling(c)
          np.random.shuffle(ages)
          np.random.shuffle(cs)

          for i in range(0,len(m[0])):
            mjack = m[:,i]
            resamples = jackknife_resampling(mjack)

            for ii in range(0,len(resamples)):
              ci = cs[ii].reshape(-1,1)
              agei = ages[ii].reshape(-1,1)
              # print(ages[ii])
              # resamplesi = np.append(resamples[ii],[1])
              resamplesi = resamples[ii].reshape(-1,1)
              combine = np.concatenate((agei,resamplesi),axis=1)
              result_coef_jackknife,result_inter_jackknife = self.Calculate_Beta_Sklearn(combine,ci)
              coeflist_jackknife.append(result_coef_jackknife)
              interlist_jackknife.append(result_inter_jackknife)
            # result = Calculate_Beta_Numpy(mi,age)
            # coeflist.append(result)
          coeflist_jackknife =np.array(coeflist_jackknife)
          interlist_jackknife =np.array(interlist_jackknife)
          return coeflist_jackknife


    def calculate_bias_interval(self,array,percent):
        lower = (1-percent)/2
        upper = 1-lower
        length = array.size
        lower_index = math.ceil((length *lower))
        upper_index = math.floor((length *upper))

        return array[lower_index:upper_index]

    def m1_zscore(self,coeflist):
        z = stats.zscore(coeflist)
        return z


    def print_result(self,m,age,c,CI_percentage,resample_time):
          result_bootstrap = self.m2_bootstrap(m,age,c)
          c1_bootstrap = result_bootstrap[:,:,:,0].ravel()
          c2_bootstrap = result_bootstrap[:,:,:,1].ravel()
          result_jackknife = self.m2_jackknife(m,age,c)
          c1_jackknife = result_jackknife[:,:,0].ravel()
          c2_jackknife = result_jackknife[:,:,1].ravel()

          CI_c1_bootstrap = self.calculate_bias_interval(np.sort(c1_bootstrap),CI_percentage)
          CI_c2_bootstrap = self.calculate_bias_interval(np.sort(c2_bootstrap),CI_percentage)

          CI_c1_jackknife = self.calculate_bias_interval(np.sort(c1_jackknife),CI_percentage)
          CI_c2_jackknife = self.calculate_bias_interval(np.sort(c2_jackknife),CI_percentage)

          zscore_c1_bootstrap = self.m1_zscore(CI_c1_bootstrap)
          zscore_c2_bootstrap = self.m1_zscore(CI_c2_bootstrap)

          zscore_c1_jackknife = self.m1_zscore(CI_c1_jackknife) 
          zscore_c2_jackknife = self.m1_zscore(CI_c2_jackknife)

          print("\n The resampling result of c1 Bootstrap is: " )
          print(c1_bootstrap)
          print("\n The resampling result of c2 Bootstrap is: " )
          print(c2_bootstrap)

          print("\n The resampling result of c1 Jackknife is: "  )
          print(c1_jackknife)
          print("\n The resampling result of c2 Jackknife is: "  )
          print(c2_jackknife)


          print("\n The confidence interval of c1 Bootstrap is: "  )
          print(CI_c1_bootstrap)
          print("\n The confidence interval of c2 Bootstrap is: "  )
          print(CI_c2_bootstrap)

          print("\n The confidence interval of c1 Jackknife is: "  )
          print(CI_c1_jackknife)
          print("\n The confidence interval of c2 Jackknife is: "  )
          print(CI_c2_jackknife)

          print("\n The Z score of c1 Bootstrap is: " )
          print(zscore_c1_bootstrap)
          print("\n The Z score of c2 Bootstrap is: " )
          print(zscore_c2_bootstrap)


          print("\n The Z score of c1 Jackknife is: " )
          print(zscore_c1_jackknife)
          print("\n The Z score of c2 Jackknife is: " )
          print(zscore_c2_jackknife)
          return c1_bootstrap,c2_bootstrap,c1_jackknife,c2_jackknife,CI_c1_bootstrap,CI_c2_bootstrap,CI_c1_jackknife,CI_c2_jackknife,zscore_c1_bootstrap,zscore_c2_bootstrap,zscore_c1_jackknife,zscore_c2_jackknife




Now, let's generate a 100 * 100 random array and run this array in module 1 and module 2 with 1000 times of resampling. The running time for model 1 and model 2 is shown down below

In [0]:
import time
m,age,c = Generate_X_y_c(100,100)
resample_time = 1000


In [63]:
start=time.time()
m1 = model_1(m,age,0.95,resample_time)
m1.print_result(m,age,0.95,resample_time)
end=time.time()
b1_bootstrap,b1_jackknife,CI_b1_bootstrap,CI_b1_jackknife,zscore_b1_bootstrap,zscore_b1_jackknife = m1.print_result(m,age,0.95,resample_time)



 The resampling result of b1 Bootstrap is: 
[-0.0377674  -0.05404626 -0.08609798 ... -0.01790599 -0.00053648
 -0.00275603]

 The resampling result of b1 Jackknife is: 
[-0.03549434 -0.0340648  -0.03262327 ... -0.00175765 -0.00315868
 -0.00352729]

 The confidence interval of b1 Bootstrap is: 
[-0.16612859 -0.16609302 -0.16604996 ...  0.13828151  0.13828483
  0.13830348]

 The confidence interval of b1 Jackknife is: 
[-0.15454908 -0.15331122 -0.15292364 ...  0.02049027  0.02049757
  0.02055091]

 The Z score of b1 Bootstrap is: 
[-4.73720958 -4.73617875 -4.73493077 ...  4.08584299  4.08593928
  4.08647988]

 The Z score of b1 Jackknife is: 
[-7.00036599 -6.94060108 -6.92188855 ...  1.45068559  1.45103806
  1.45361323]

 The resampling result of b1 Bootstrap is: 
[-0.0377674  -0.05404626 -0.08609798 ... -0.01790599 -0.00053648
 -0.00275603]

 The resampling result of b1 Jackknife is: 
[-0.03549434 -0.0340648  -0.03262327 ... -0.00175765 -0.00315868
 -0.00352729]

 The confidence interva

In [64]:
print('Total model 1 running time: %s Seconds'%(end-start))

Total model 1 running time: 66.10864782333374 Seconds


In [65]:
start=time.time()
m2 = model_2(m,age,c,0.95,resample_time)
m2.print_result(m,age,c,0.95,resample_time)
end=time.time()


 The resampling result of c1 Bootstrap is: 
[ 0.11401927  0.14909964  0.00624501 ... -0.12020291  0.16351532
  0.22286234]

 The resampling result of c2 Bootstrap is: 
[-0.01686165 -0.04164814 -0.03782181 ... -0.01414499  0.00886081
 -0.00267617]

 The resampling result of c1 Jackknife is: 
[-0.00559518 -0.03940203  0.05491778 ...  0.11181291  0.01375255
  0.01132781]

 The resampling result of c2 Jackknife is: 
[ 0.02952425 -0.01558528 -0.02479599 ...  0.00542782  0.00523557
  0.00354991]

 The confidence interval of c1 Bootstrap is: 
[-0.20576786 -0.20576786 -0.20576786 ...  0.19403583  0.19403583
  0.19403583]

 The confidence interval of c2 Bootstrap is: 
[-0.15693514 -0.15692439 -0.15691027 ...  0.15237956  0.15239972
  0.15242866]

 The confidence interval of c1 Jackknife is: 
[-0.13555287 -0.13555287 -0.13555287 ...  0.14038269  0.14038269
  0.14038269]

 The confidence interval of c2 Jackknife is: 
[-0.0770167  -0.07696122 -0.07611405 ...  0.07569783  0.07634142
  0.07651347]


In [66]:
print('Total model 2 running time: %s Seconds'%(end-start))

Total model 2 running time: 72.90235161781311 Seconds


Finally, let's multiply b1 and c2 in all the sections. The total calculation time is shown down below. 

In [67]:
c1_bootstrap,c2_bootstrap,c1_jackknife,c2_jackknife,CI_c1_bootstrap,CI_c2_bootstrap,CI_c1_jackknife,CI_c2_jackknife,zscore_c1_bootstrap,zscore_c2_bootstrap,zscore_c1_jackknife,zscore_c2_jackknife = m2.print_result(m,age,c,0.95,resample_time)
#multiply them
start=time.time()
b1_c2_bootstrap = b1_bootstrap*c2_bootstrap
print("The result of b1*c2 by bootstrap is:")
print(b1_c2_bootstrap)

b1_c2_jackknife = b1_jackknife*c2_jackknife
print("The result of b1*c2 by jackknife is:")
print(b1_c2_jackknife)

b1_c2_ci_bootstrap = CI_b1_bootstrap*CI_c2_bootstrap
print("The result of b1*c2 confidence interval by bootstrap is:")
print(b1_c2_ci_bootstrap)

b1_c2_ci_jackknife = CI_b1_jackknife*CI_c2_jackknife
print("The result of b1*c2 confidence interval by jackknife is:")
print(b1_c2_ci_jackknife)

b1_c2_zscore_bootstrap = zscore_b1_bootstrap*zscore_c1_bootstrap
print("The result of b1*c2 zscore by bootstrap is:")
print(b1_c2_zscore_bootstrap)

b1_c2_zscore_jackknife = zscore_b1_jackknife*zscore_c1_jackknife
print("The result of b1*c2 zscore by jackknife is:")
print(b1_c2_zscore_jackknife)
end=time.time()



 The resampling result of c1 Bootstrap is: 
[ 0.11401927  0.14909964  0.00624501 ... -0.12020291  0.16351532
  0.22286234]

 The resampling result of c2 Bootstrap is: 
[-0.01686165 -0.04164814 -0.03782181 ... -0.01414499  0.00886081
 -0.00267617]

 The resampling result of c1 Jackknife is: 
[-2.01547614e-02 -2.38521369e-05 -1.14175954e-01 ... -1.23347600e-01
  1.12981070e-01 -1.47784112e-01]

 The resampling result of c2 Jackknife is: 
[-0.00370558  0.01947016  0.00075126 ...  0.00575551  0.00552516
  0.00189333]

 The confidence interval of c1 Bootstrap is: 
[-0.20576786 -0.20576786 -0.20576786 ...  0.19403583  0.19403583
  0.19403583]

 The confidence interval of c2 Bootstrap is: 
[-0.15693514 -0.15692439 -0.15691027 ...  0.15237956  0.15239972
  0.15242866]

 The confidence interval of c1 Jackknife is: 
[-0.13273138 -0.13273138 -0.13273138 ...  0.12297332  0.12297332
  0.12297332]

 The confidence interval of c2 Jackknife is: 
[-0.08213749 -0.08203013 -0.08170539 ...  0.0814151   0

In [68]:
print('Total b1*c2 running time: %s Seconds'%(end-start))

Total b1*c2 running time: 0.0061054229736328125 Seconds
